# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 249, done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 249
Receiving objects: 100% (249/249), 72.12 MiB | 36.29 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [3]:
!git clone https://github.com/1adrianb/face-alignment


fatal: destination path 'face-alignment' already exists and is not an empty directory.


In [4]:
%cd 'face-alignment'
%pip install -r requirements.txt
!python setup.py install

/content/face-alignment
running install
running bdist_egg
running egg_info
creating face_alignment.egg-info
writing face_alignment.egg-info/PKG-INFO
writing dependency_links to face_alignment.egg-info/dependency_links.txt
writing requirements to face_alignment.egg-info/requires.txt
writing top-level names to face_alignment.egg-info/top_level.txt
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/face_alignment
copying face_alignment/utils.py -> build/lib/face_alignment
copying face_alignment/models.py -> build/lib/face_alignment
copying face_alignment/__init__.py -> build/lib/face_alignment
copying face_alignment/api.py -> build/lib/face_alignment
creating build/lib/face_alignment/detection
copying face_alignment/detection/core.py -> build/lib/face_alignment/detection
copyin

In [5]:
!ls

build  Dockerfile  face_alignment	    README.md	      setup.py
conda  docs	   face_alignment.egg-info  requirements.txt  test
dist   examples    LICENSE		    setup.cfg	      tox.ini


In [6]:

%cd '../first-order-model'
!ls

/content/first-order-model
animate.py	 demo.ipynb	    logger.py	       run.py
augmentation.py  demo.py	    modules	       sup-mat
config		 Dockerfile	    README.md	       sync_batchnorm
crop-video.py	 frames_dataset.py  reconstruction.py  train.py
data		 LICENSE.md	    requirements.txt


In [27]:
%pip install  ffmpeg
%pip install imageio-ffmpeg

     |████████████████████████████████| 26.9MB 118kB/s 


In [28]:
!python crop-video.py --inp /content/Aiz.mp4

323it [01:24,  3.92it/s]Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/imageio/plugins/ffmpeg.py", line 621, in _read_frame_data
    "Frame is %i bytes, but expected %i." % (len(s), framesize)
RuntimeError: Frame is 0 bytes, but expected 6220800.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "crop-video.py", line 154, in <module>
    commands = process_video(args)
  File "crop-video.py", line 94, in process_video
    for i, frame in tqdm(enumerate(video)):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.6/dist-packages/imageio/core/format.py", line 405, in iter_data
    im, meta = self._get_data(i)
  File "/usr/local/lib/python3.6/dist-packages/imageio/plugins/ffmpeg.py", line 436, in _get_data
    result, is_new = self._read_frame()
  File "/usr/local/lib/python3.6/dist-packages/imageio/plugins/

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.
Alternativelly you can use this mirror link https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing**

**Load driving video and source image**

In [52]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

#source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
#reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
source_image = imageio.imread('/content/Apes_Caesar.png')
driving_video='/content/first-order-model/Aiz_New.mp4'
reader = imageio.get_reader(driving_video)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [13]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-adv-256.yaml', 
                            checkpoint_path='/content/drive/MyDrive/Motion Capture/vox-adv-cpk.pth.tar')

In [53]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 323/323 [00:13<00:00, 24.52it/s]


Insert audio into animation

In [60]:
import moviepy.editor as mpe
video_clip = mpe.VideoFileClip('/content/generated.mp4')

 #extract audio from original driving video
audio_clip = mpe.AudioFileClip('/content/first-order-model/Aiz_New.mp4')

#set audio into your output video and saving it to your google drive
final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile('/content/final_video.mp4',fps=fps, codec='libx264')

# #Put 2 clips side-by-side and saving it to your google drive
org_clip = mpe.VideoFileClip('/content/first-order-model/Aiz_New.mp4')
merge_clip = mpe.clips_array([[org_clip,final_clip]])
merge_clip.write_videofile("/content/final-out_compare.mp4",fps=fps,codec='libx264')

video_clip.close()
audio_clip.close()
final_clip.close()
org_clip.close()
merge_clip.close()
#HTML(display(source_image, driving_video, '/content/final_video.mp4').to_html5_video())

[MoviePy] >>>> Building video /content/final_video.mp4
[MoviePy] Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 240/240 [00:00<00:00, 737.57it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/final_video.mp4



100%|██████████| 324/324 [00:01<00:00, 310.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/final_video.mp4 

[MoviePy] >>>> Building video /content/final-out_compare.mp4
[MoviePy] Writing audio in final-out_compareTEMP_MPY_wvf_snd.mp3


100%|██████████| 240/240 [00:00<00:00, 383.17it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/final-out_compare.mp4



100%|██████████| 326/326 [00:02<00:00, 110.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/final-out_compare.mp4 



In [49]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 323/323 [00:13<00:00, 23.64it/s]


**Perform image animation**

In [47]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 323/323 [00:13<00:00, 24.69it/s]


In [32]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 323/323 [00:12<00:00, 25.46it/s]


**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [38]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [43]:
!ffmpeg -i /content/Aiz.mp4 -ss 00 -t 10:00 -filter:v "crop=975:975:525:0, scale=256:256" -async 1 Aiz_New.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/09.png')
driving_video = imageio.mimread('hinton.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 240/240 [00:30<00:00,  7.94it/s]
